In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'ESAB',
       'FCFS', 'GEN', 'HUBB', 'IBKR', 'ISEE', 'JJC', 'LSI', 'PDCE', 'PEAK',
       'PFG', 'ROCC', 'SBNY', 'SRPT', 'UA', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VNDA', 'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=227)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'AJRD', 'ALGM', 'BJ', 'BRBR', 'CARR', 'CDAY', 'CEG',
       'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DCP', 'DDOG', 'DOW', 'DT',
       'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FISV', 'FOX', 'FOXA', 'FYBR', 'GEHC',
       'GFS', 'GO', 'GXO', 'ISEE', 'JJC', 'KD', 'LCID', 'LSI', 'MP', 'MRNA',
       'NARI', 'NVST', 'NVT', 'OGN', 'OTIS', 'PDCE', 'PDD', 'PGNY', 'RE',
       'RIVN', 'ROCC', 'SGOV', 'SHC', 'SITM', 'SWAV', 'UNVR', 'VICI', 'VNT',
       'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 292


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-08-08,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['SHV', 'STRL']",['SMCI'],"['BCC', 'LOB', 'TDS']","['CLS', 'FTSM', 'SKYW']","['BKU', 'EME', 'GBX', 'KDNY', 'SATS']","['AMPH', 'BZH', 'CVBF', 'ELF', 'EXTR', 'FFWM']",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],"['SHV', 'STRL']",['SMCI'],"['BCC', 'LOB', 'TDS']","['CLS', 'FTSM', 'SKYW']","['BKU', 'EME', 'GBX', 'KDNY', 'SATS']","['AMPH', 'BZH', 'CVBF', 'ELF', 'EXTR', 'FFWM']",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 3840
unique symbol count from model picks: 205


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['AJRD', 'CHT', 'CVLT', 'DCP', 'DVA', 'HY', 'ISEE', 'PKX', 'SRPT', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 292
len(common_symbols): 10
len(symbols_drop): 282


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 282


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      27
symbols with same volume:   227
symbols with no close:       61

symbols total before drop:                                        1,578
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   282

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,296    1,500
df Close (df_c) after dropped symbols:      1,296    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-08-03  127.300003  127.360001  124.970001  125.050003  1742600.0   
2023-08-04  125.440002  127.169998  125.080002  126.300003  1819500.0   
2023-08-07  126.730003  127.889999  125.699997  126.510002  1487000.0   
2023-08-08  126.690002  128.630005  125.070000  128.350006  2525200.0   
2023-08-09  128.160004  129.419998  127.529999  127.629997  1786049.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-08-03  33.899853  34.587819  33.411295  34.398380  4243300.0  ...   
2023-08-04  34.448233  34.827115  33.655577  33.850002  3815100.0  ...   
2023-08-07  33.900002  34.060001  33.220001  34.060001  3520600.0  ...   
2023-08-08  33.020000  33.349998  32.680000  33.230000  5366300.0  ...   
2023-08-09  33.480000  34.090000  33.029999  33.619999  3919156.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-08-03  18.820000  19.180000  18.730000  19.030001  138100.0  30.129999   
2023-08-04  19.059999  19.059999  18.520000  18.719999  254000.0  30.000000   
2023-08-07  18.730000  18.790001  18.350000  18.709999  412000.0  29.940001   
2023-08-08  18.580000  18.700001  18.209999  18.360001  259200.0  29.980000   
2023-08-09  18.389999  18.615000  17.930000  18.059999  243897.0  29.570000   

                                                       
                 High        Low      Close    Volume  
Date                                                   
2023-08-03  30.280001  29.809999  30.160000  690600.0  
2023-08-04  30.430000  29.740000  29.870001  950500.0  
2023-08-07  30.490000  29.940001  30.270000  754000.0  
2023-08-08  29.980000  29.420000  29.910000  631200.0  
2023-08-09  29.900000  29.490000  29.520000  599654.0  

[5 rows x 6480 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABC,ABM,ABR,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-03,125.050003,34.398380,15.98,72.029999,191.169998,31.840000,149.050003,191.360001,45.419998,16.480000,...,33.980000,125.790001,249.550003,71.800003,53.549999,37.410000,27.030001,180.880005,19.030001,30.160000
2023-08-04,126.300003,33.850002,15.84,71.010002,181.990005,31.590000,147.729996,185.330002,45.560001,16.709999,...,33.860001,126.320000,251.410004,68.730003,54.770000,37.919998,26.790001,178.630005,18.719999,29.870001
2023-08-07,126.510002,34.060001,15.99,71.300003,178.850006,31.040001,150.330002,186.580002,45.599998,16.740000,...,33.240002,125.629997,248.229996,69.629997,54.520000,38.139999,26.770000,180.690002,18.709999,30.270000
2023-08-08,128.350006,33.230000,16.18,70.559998,179.800003,30.340000,149.619995,185.369995,45.880001,16.510000,...,32.750000,124.860001,253.679993,69.059998,55.310001,38.180000,26.600000,189.300003,18.360001,29.910000
2023-08-09,127.629997,33.619999,15.76,71.199997,178.190002,29.400000,150.770004,185.710007,46.439999,16.510000,...,32.619999,125.209999,257.890015,67.160004,54.869999,37.400002,26.320000,189.350006,18.059999,29.520000
